In [2]:
from cloudvolume import Skeleton, CloudVolume
import numpy as np 

/Users/thomasathey/Documents/mimlab/mouselight/env/lib/python3.8/site-packages/python_jsonschema_objects/__init__.py:50: UserWarning: Schema version http://json-schema.org/draft-04/schema not recognized. Some keywords and features may not be supported.
  warnings.warn(


## Read local swc

In [64]:
path = "/Users/thomasathey/Documents/mimlab/mouselight/solange/Neuron_1_axon_consensus_final.swc"
skel = Skeleton.from_swc(path)

ValueError: not enough values to unpack (expected 7, got 1)

## Read remote skeleton

In [3]:
vol = CloudVolume('precomputed://https://open-neurodata.s3.amazonaws.com/brainlit/brain2_segments')
skel1 = vol.skeleton.get(1)
skel101 = vol.skeleton.get(101)

skel101.id = 1
print(f"Equivalent skeletons?: {Skeleton.equivalent(skel1, skel101)}")

Downloading: 100%|██████████| 1/1 [00:00<00:00,  4.45it/s]Equivalent skeletons?: False



In [4]:
print(skel101)
print(skel1)
print((skel101.vertices == skel1.vertices).all())
print((skel101.edges == skel1.edges).all())
print((skel101.radius == skel1.radius).all())
print((skel101.vertex_types == skel1.vertex_types).all())
print(len(skel101.vertex_types == skel1.vertex_types))
print(np.sum(skel101.vertex_types == skel1.vertex_types))
print(skel101.vertex_types[skel101.vertex_types != skel1.vertex_types])
print(skel1.vertex_types[skel101.vertex_types != skel1.vertex_types])
skel101.id = 101

Skeleton(segid=1, vertices=(shape=2626, float32), edges=(shape=2625, uint32), radius=(2626, float32), vertex_types=(2626, uint8), space='physical' transform=[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0]])
Skeleton(segid=1, vertices=(shape=2626, float32), edges=(shape=2625, uint32), radius=(2626, float32), vertex_types=(2626, uint8), space='physical' transform=[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0]])
True
True
True
False
2626
2366
[  0   0   0   0   0   0  32  67 128  66   0   0   0   0   0   0  32  67
 128  66  32  67 128  66   0   0  32  67 128  66   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0  32  67 128  66   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  32  67
 128  66   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  32  67 128  66   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0

## Upload skel file with cv

In [59]:
skel1.id = 101
segid = skel1.id

UnsupportedProtocolError: [1;93m
Cloud Path must conform to FORMAT://PROTOCOL://BUCKET/PATH
Examples: 
  precomputed://gs://test_bucket/em
  gs://test_bucket/em
  graphene://https://example.com/image/em

Supported Formats: None (precomputed), graphene, precomputed, boss
Supported Protocols: gs, file, s3, matrix, http, https

Cloud Path Recieved: .
[m

In [53]:
vol.skeleton.upload(skel1)

Uploading:   0%|          | 0/1 [00:00<?, ?it/s]


NotImplementedError: 

## Write binary file manually

In [62]:
import sys
sys.byteorder

'little'

Encoded skeleton file format
The skeleton representation for a single segment ID is a binary file with the following format:
num_vertices: uint32le, specifies the number of vertices.
num_edges: uint32le, specifies the number of edges.
vertex_positions: 3*num_vertices float32le, as a C-order [num_vertices, 3] array specifying the x, y, and z vertex positions in "stored model" coordinates.
edges: 2*num_edges uint32le, as a C-order [num_edges, 2] array specifying the source and target vertex index in the range [0, num_vertices).
For each additional attribute in vertex_attributes:
attribute_data: num_vertices * num_components elements of the specified data_type in little-endian format.

In [5]:
import struct
from io import BytesIO

In [61]:
skel = skel1
skel_bio = BytesIO()

# write the number of vertices and edges
skel_bio.write(struct.pack('<I',skel.vertices.shape[0]))
skel_bio.write(struct.pack('<I',skel.edges.shape[0]))

# write the actual vertices and edges
#s = struct.pack(f'<{len(skel.vertices)}f',*(skel.vertices.tolist()))

#verify that numpy is writing the right format
skel_bio.write(skel.vertices.astype('float32').tobytes())
skel_bio.write(skel.edges.astype('uint32').tobytes())

# write the attributes
for i in range(len(skel.radius)):
     skel_bio.write(skel.radius[i].astype('float32').tobytes())

for i in range(len(skel.vertex_types)):
    skel_bio.write(skel.vertex_types[i].astype('float32').tobytes()) 

# write the file
with open('/Users/thomasathey/Documents/mimlab/mouselight/solange/101', 'wb') as fp:
    fp.write(skel_bio.getbuffer())

In [93]:
import struct
from io import BytesIO
# assume you have a skeleton object already
# called `skel`

skel = skel1
skel.extra_attributes[-1]["data_type"] = "float32"
skel_bio = BytesIO()
# write the number of vertices and edges
skel_bio.write(struct.pack('<I',skel.vertices.shape[0]))
skel_bio.write(struct.pack('<I',skel.edges.shape[0]))
# write the actual vertices and edges
skel_bio.write(struct.pack(f'<{skel.vertices.size}f',*(skel.vertices.flatten().tolist())))
skel_bio.write(struct.pack(f'<{skel.edges.size}I',*(skel.edges.flatten().tolist())))
# write the attributes
for i in range(len(skel.radius)):
     skel_bio.write(struct.pack('<f',skel.radius[i].tolist()))
for i in range(len(skel.vertex_types)):
    skel_bio.write(struct.pack('<f',float(skel.vertex_types[i])))
# write the file
with open('/Users/thomasathey/Documents/mimlab/mouselight/solange/101', 'wb') as fp:
    fp.write(skel_bio.getbuffer())

In [94]:
skel.extra_attributes

[{'id': 'radius', 'data_type': 'float32', 'num_components': 1},
 {'id': 'vertex_types', 'data_type': 'float32', 'num_components': 1}]

In [22]:
skel.radius[0]

1.0

In [26]:
skel

Skeleton(segid=101, vertices=(shape=2626, float32), edges=(shape=2625, uint32), radius=(2626, float32), vertex_types=(2626, uint8), space='physical' transform=[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0]])